In [2]:
import os

import zipfile as zip
import pandas as pd
import csv
import numpy as np
import requests as rq
import io
from scipy import optimize
import matplotlib.pyplot as plt
import requests, pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import pip
import warnings
from cif import cif
import xml.etree.ElementTree as ET
import pandas_read_xml as pdx
import pandasdmx as sdmx
import re

C:\Users\vw191\Downloads\annaconda\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
C:\Users\vw191\Downloads\annaconda\lib\site-packages\pandasdmx\remote.py:11: RuntimeWarning: optional dependency requests_cache is not installed; cache options to Session() have no effect
  warn(


In [58]:
df = pd.read_excel(r'C:\Users\vw191\OneDrive - University of Bristol\Course\3rd year\Data Science\Project\Datastore\oecd.xlsx', sheet_name='OECD.Stat export',header=None,skiprows=6,nrows=1132,na_values='..')
df.columns = ['metro_area','year','unit','pop_density','pop_density_core','pop_density_hinterland','urbanised_area_per_capita','polycentricity','pop','pop_core','pop_hinterland','gr_index_pop','pop_metro_shr_cry','gdp_metro_shr_cry','gdp_per_capita','prod','gini','poverty_rate','shr_access_fibre','homocide_rate','shr_pop_core_pm2.5>25','tree_cover_shr_core_area'] #Rename all columns.
df = df.drop('unit',axis=1) #Drop unit column.
df.metro_area = df.metro_area.ffill(axis=0,limit=11) #As the file was xlsx the metropolitan names did not fill for each row. Therefore I ffill to include the correct name for all rows.
df.polycentricity = df.polycentricity.bfill(axis=0,limit=11) #The file includes polycentricity dummy variable data only in the last year for each city. Therefore, I bfill to include the polycentricity data for all rows.
df[['id','name']] = df['metro_area'].str.split(':', 1, expand=True) #split metro_area column into the oecd id and metro name.
df.head(26)
df1 = df

In [59]:
df.head(23)

,metro_area,year,pop_density,pop_density_core,pop_density_hinterland,urbanised_area_per_capita,polycentricity,pop,pop_core,pop_hinterland,...,gdp_per_capita,prod,gini,poverty_rate,shr_access_fibre,homocide_rate,shr_pop_core_pm2.5>25,tree_cover_shr_core_area,id,name
0,AUS01: Greater Sydney,2010,367.0,2008.0,56.0,NaN,1.0,4555516.0,3974090.0,581426.0,...,48234.0,101356.0,NaN,NaN,NaN,NaN,0.0,NaN,AUS01,Greater Sydney
1,AUS01: Greater Sydney,2011,372.0,2032.0,56.0,NaN,1.0,4608949.0,4021757.0,587192.0,...,48261.0,100052.0,NaN,NaN,NaN,NaN,0.0,NaN,AUS01,Greater Sydney
2,AUS01: Greater Sydney,2012,377.0,2063.0,57.0,NaN,1.0,4677196.0,4082807.0,594389.0,...,47949.0,100370.0,NaN,NaN,NaN,NaN,NaN,NaN,AUS01,Greater Sydney
3,AUS01: Greater Sydney,2013,384.0,2099.0,58.0,NaN,1.0,4757364.0,4154747.0,602617.0,...,47483.0,99396.0,NaN,NaN,NaN,NaN,0.0,NaN,AUS01,Greater Sydney
4,AUS01: Greater Sydney,2014,390.0,2136.0,59.0,286.0,1.0,4841349.0,4228797.0,612552.0,...,47523.0,100689.0,NaN,NaN,NaN,NaN,0.0,NaN,AUS01,Greater Sydney
5,AUS01: Greater Sydney,2015,398.0,2175.0,60.0,NaN,1.0,4930189.0,4305711.0,624478.0,...,47430.0,100267.0,NaN,NaN,NaN,NaN,0.0,22.5,AUS01,Greater Sydney
6,AUS01: Greater Sydney,2016,405.0,2216.0,61.0,NaN,1.0,5024923.0,4385372.0,639551.0,...,48416.0,100611.0,NaN,NaN,NaN,NaN,0.0,NaN,AUS01,Greater Sydney
7,AUS01: Greater Sydney,2017,414.0,2264.0,63.0,NaN,1.0,5136919.0,4481383.0,655536.0,...,47954.0,100947.0,NaN,NaN,NaN,NaN,0.0,NaN,AUS01,Greater Sydney
8,AUS01: Greater Sydney,2018,421.0,2300.0,65.0,NaN,1.0,5225098.0,4552448.0,672650.0,...,48219.0,99601.0,NaN,NaN,NaN,NaN,0.0,22.4,AUS01,Greater Sydney
9,AUS01: Greater Sydney,2019,428.0,2334.0,66.0,NaN,1.0,5310099.0,4620452.0,689647.0,...,47596.0,96743.0,NaN,NaN,NaN,NaN,0.0,NaN,AUS01,Greater Sydney
